# Parameter Sweeps - Second Attempt

In [156]:
import ipyparallel as ipp
import ipywidgets as ipw
import pandas as pd
import qgrid
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pymongo
import itertools as it
import warnings

In [2]:
import time

In [50]:
rc = ipp.Client()
rc.ids

/home/oliver/local/miniconda3/envs/widgets/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@oliver-arch')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


[0, 1]

In [4]:
# Tell engines their ids
for i, engine in enumerate(rc):
    engine.push({'engine_id': i})

# Ask them to report back
%px print(engine_id)

[stdout:0] 0
[stdout:1] 1


In [5]:
def printme(*args, **kwargs):
    return engine_id, args, kwargs

In [6]:
lview = rc.load_balanced_view()

In [7]:
lview.apply(printme, 1, 3, x=4, y=79.1).result()

(1, (1, 3), {'x': 4, 'y': 79.1})

In [8]:
db = pymongo.MongoClient('nautilus.optiputer.net', 31017)['kale-param-sweeps']
coll = db['sweep1']

In [9]:
%%time
# Test execution scheduling
def printsleep(dur, msg):
    import time
    time.sleep(dur)
    return '{}: {}'.format(engine_id, msg)

futures = [lview.apply(printsleep, 2, 'hello') for i in range(3)]
res = [f.result() for f in futures]

CPU times: user 25.9 ms, sys: 2.83 ms, total: 28.7 ms
Wall time: 4.04 s


# Kale functions

## View

In [10]:
def visualize_wrapper(fun, output, *args, **kwargs):
    """Call visualization function and capture output"""
    # Avoid using output context to ensure that
    # only this function's output is included.
    @output.capture(clear_output=True, wait=True)
    def wrapper():
        fun(*args, **kwargs)
    wrapper()

In [11]:
def param_grid(**kwargs):
    """Generate Cartesian product of keyword arguments"""
    prod = it.product(kwargs.items())
    name_list, values_list = zip(*kwargs.items())
    value_combinations = list(it.product(*values_list))
    return name_list, value_combinations

def param_df(**kwargs):
    """Generate pandas DataFrame from Cartesian product of keyword arguments"""
    names, values = param_grid(**kwargs)
    return pd.DataFrame(values, columns=names)

def param_qgrid(qgrid_layout=None, **kwargs):
    """Generate Qgrid table from Cartesian product of keyword arguments"""
    if not qgrid_layout:
        qgrid_layout=ipw.Layout()
    return qgrid.QgridWidget(df=param_df(**kwargs), layout=qgrid_layout)

In [12]:
g = param_qgrid(qgrid_layout=ipw.Layout(width='300px'), x=[1,2], y=[6,3], z=[True, False])
g

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Submit

In [13]:
def dict_list_from_df(df):
    """Turn each row into a dictionary indexed by column names"""
    return [{col: val for col, val in zip(df.columns, df.loc[ind,:])} for ind in df.index]

In [167]:
#class ParamSpan(tr.HasTraits):
#    pass

class ParamSpanWidget(ipw.VBox):
    def __init__(self, param_span_name, compute_func, vis_func, **all_params):
        self.name = param_span_name
        self.all_params = all_params
        self.compute_func = compute_func
        self.vis_func = vis_func
        
        self.init_engines()
        self.init_widgets()
        self.init_logic()
        self.init_layout()
    
    def init_engines(self):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.lview = ipp.Client().load_balanced_view()
        
    def init_widgets(self):
        output_layout = ipw.Layout(height='300px', border='1px solid', overflow_x='scroll', overflow_y='scroll')
        qgrid_layout = ipw.Layout()
        self.output = ipw.Output(layout=output_layout)
        self.param_table = param_qgrid(qgrid_layout, **self.all_params)
        
    def compute_and_visualize(self, *change):
        """TODO: Separate these, add MongoDB in the middle"""
        # Get parameters
        # Only take first
        params = dict_list_from_df(self.param_table.get_selected_df())[0]
        compute_result = self.lview.apply(self.compute_func, **params).result()
        visualize_wrapper(self.vis_func, self.output, **compute_result)
    
    def init_logic(self):
        self.param_table.observe(self.compute_and_visualize, names='_selected_rows')
    
    def init_layout(self):
        super().__init__([
            self.output,
            self.param_table
        ])

# User functions

In [158]:
def compute1(word, number):
    return dict(
        engine_id = engine_id,
        concat = '{} <3 {}'.format(word, number),
        doubled = 2*word,
        squared = number**2
    )

In [159]:
def visualize1(engine_id, concat, doubled, squared):
    print("These are the results.")
    print("Engine: {}".format(engine_id))
    print("Catted '{}'".format(concat))
    print("doubled '{}' and squared '{}'".format(doubled, squared))

In [160]:
psw1 = ParamSpanWidget('test_span', compute1, visualize1, word=['hi', 'bye'], number=[1,2,3])
psw1

ParamSpanWidget(children=(Output(layout=Layout(border='1px solid', height='300px', overflow_x='scroll', overfl…

# Normal Distribution Plots

In [165]:
def compute2(N, mean, std, color):
    import numpy as np
    x = np.random.normal(loc=mean, scale=std, size=N)
    realmean = np.mean(x)
    realstd = np.std(x)
    return {
        'engine_id': engine_id,
        'x': x,
        'N': N,
        'realmean': realmean,
        'realstd': realstd,
        'color': color
    }
    
def visualize2(engine_id, x, N, realmean, realstd, color):
    print("Computed on engine {}".format(engine_id))
    print("Data: {}".format(x))
    plt.figure(figsize=[8,3])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.distplot(x, color=color)
    plt.title(r'$N={}$, $\mu={:.2f}$, $\sigma={:.2f}$'.format(N, realmean, realstd))
    plt.show()

In [166]:
psw2 = ParamSpanWidget('test_span', compute2, visualize2, N=[5,10,20], mean=[0,1,2], std=[5,10], color=['red', 'blue'])
psw2

ParamSpanWidget(children=(Output(layout=Layout(border='1px solid', height='300px', overflow_x='scroll', overfl…